# Day 1

## Part I

首先读取输入文件，此处为了简单起见，直接使用了Pandas的IO函数将输入数据读取到一个DataFrame当中，因为该DataFrame只有一列，因此将其取出成Serie，最后转换成Numpy的ndarray返回。

In [1]:
import numpy as np
import pandas as pd
def read_input() -> np.ndarray:
    serie = pd.read_csv('input.txt', header=None)
    return serie[0].to_numpy()

对于第一部分，最简单的方式是暴力枚举，下面用了普通for loop的方式，也可以使用itertools.combinations，代码不列。

In [2]:
def q1_solution1(serie: np.ndarray) -> int:
    for i in range(len(serie)-1):
        for j in range(i+1, len(serie)):
            if serie[i] + serie[j] == 2020:
                return serie[i] * serie[j]
    return -1

如果该数列是有序的，则可采用双指针的方式，提升性能，即使加上排序的耗时，应该也比上面的方法要快。

In [3]:
def q1_solution2(serie: np.ndarray) -> int:
    serie.sort()
    i, j = 0, len(serie)-1 # i, j 双指针，初始化指向数列首尾
    while i < j:
        # 如果找到，返回两者乘积
        if serie[i] + serie[j] == 2020:
            return serie[i] * serie[j]
        # 如果两者之和小于2020，将i指针向后移动一位
        if serie[i] + serie[j] < 2020:
            i += 1
        # 如果两者之和大于2020，将j指针向前移动一位
        if serie[i] + serie[j] > 2020:
            j -= 1
    return -1

In [4]:
s = read_input()
s

array([1440, 1511, 1731, 1400, 1542, 1571, 1768, 1730, 1959, 1342, 1744,
        872, 1237, 1846, 1597, 1583, 1711, 1499, 1679, 1895, 1875, 1928,
       1728, 1673,  481, 1934,  673, 1704, 1916, 1958, 1821, 1649, 1640,
       1802, 1732,  121, 1924, 1438, 1748, 1046, 1905, 1566, 1152, 1964,
       1518, 1603, 1414, 1785, 1993, 1594, 1761, 1455, 1738, 1699, 1507,
       1483, 1450, 1653, 1644,   19, 1340, 1227, 1353, 2009, 1188, 1228,
       1898, 1941, 1515, 1766, 1351, 1980, 1378, 1702, 1620, 1729, 1279,
       1384, 1894, 1770, 1853, 1161, 1970, 1986, 1669, 1938, 1602, 1190,
       1822,  425, 1750, 1632, 1613, 1805, 1718, 1990, 1762, 1242, 1485,
       1598, 1893, 1995, 1823, 1786, 1506, 1464, 1467, 1639, 1674, 1903,
       1961, 1478, 1847, 1760, 1997, 2010,  899, 2000, 1488, 1243, 1891,
       1504, 1693, 1176, 1391, 1563,  692, 1497, 1428, 1745, 1368, 1723,
       1989, 1930, 1171, 1840, 1372, 1987, 1952, 1842, 1967, 1759, 1929,
       1945, 1919, 1333, 1692, 1811, 1221, 1520, 19

In [5]:
q1_solution1(s)

1013211

In [6]:
q1_solution2(s)

1013211

OK！两种方法算得同样结果。

下面来看看耗时。

In [7]:
s = read_input()

%timeit q1_solution1(s)

8.92 ms ± 124 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%timeit q1_solution2(s)

288 µs ± 5.79 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


约相差了30倍的效率。

## Part II

因为此时数列已经是排序好的，所以我们可以简单的使用一个二分查找来搜索一个元素是否存在，定义下面这个帮助工具函数：

In [9]:
def binary_search_in(arr: np.ndarray, value: int) -> bool:
    if len(arr) == 0 or (len(arr) == 1 and arr[0] != value):
        return False
    mid = len(arr) // 2
    if value == arr[mid]:
        return True
    if value < arr[mid]:
        return binary_search_in(arr[:mid], value)
    return binary_search_in(arr[mid+1:], value)

简单检验一下这个函数是否正常工作：

In [10]:
assert(binary_search_in(s, 481))
assert(binary_search_in(s, 1725))
assert(not binary_search_in(s, 2020))

下面来实现第二部分问题，同样是双指针，但是因为是三个数相加，因此需要嵌套一层循环，该循环实现成从尾指针向首指针移动，并且只会搜索两个指针之间的部分元素，进一步提升性能：

In [11]:
def q2_solution(serie: np.ndarray) -> int:
    i, j = 0, len(serie)-1 # 双指针
    while i<j:
        # 当首尾指针相加不小于2020时，尾指针一直向前移动直至满足
        while serie[i] + serie[j] >= 2020:
            j -= 1
        # 嵌套循环，从尾指针开始向首指针移动，直到碰到合适的元素为止
        for k in range(j, i+1, -1):
            rem = 2020 - serie[i] - serie[k]
            if rem > serie[k]:
                break
            # 二分搜索，仅搜索i，k两个指针之间的部分数列即可
            if binary_search_in(serie[i+1:k], rem):
                return serie[i] * serie[k] * rem
        i += 1
    return -1

In [12]:
q2_solution(s)

13891280

看一下耗时情况：

In [13]:
%timeit q2_solution(s)

808 µs ± 99.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


So far so good!